In [1]:
from platform import system

from Demos.OpenEncryptedFileRaw import dst_dir

print("OK")

OK


In [25]:
%pwd

'C:\\Python_AI\\MediAI'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'C:\\Python_AI\\MediAI'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data From PDF
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()  # Fix the variable name here
    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [16]:
#Cracted_Chunks
def text_split(extracted_data):
      text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
      text_chunks=text_splitter.split_documents(extracted_data)
      return text_chunks

In [7]:
text_chunks=text_split(extracted_data)
print("Length of the Chunks",len(text_chunks))

NameError: name 'text_split' is not defined

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
#Downloading the Embeddings
def download_hugging_face_embeddings():
        embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        return embeddings

In [11]:
embeddings=download_hugging_face_embeddings()

C:\Users\nidhi\AppData\Local\Temp\ipykernel_20676\3311451588.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\nidhi\.conda\envs\MediAI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
#query_result= embeddings.embed_query("Hi I am Nidhin")
#print("Length",len(query_result))

Length 384


In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
PINECONE_AIP_KEY = os.environ.get('PINECONE_AIP_KEY')
OPEN_AI_API_KEY = os.environ.get('OPEN_AI_API_KEY')

In [26]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_AIP_KEY)

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '0d1cf92abf7a99e3c5fdf467020d0e4b', 'date': 'Tue, 27 May 2025 18:12:32 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [35]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_AIP_KEY
os.environ["OPEN_AI_API_KEY"] = OPEN_AI_API_KEY

In [28]:
#Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,)

KeyboardInterrupt: 

In [29]:
#Load Existing INdex
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [30]:
docsearch

In [31]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [32]:
retrieved_docs = retriever.invoke("What is Acne?")

In [33]:
retrieved_docs

[Document(id='31de3b7d-8dd6-4a63-8f40-a475ed1e09ef', metadata={'page': 3353.0, 'source': 'Data\\Encyclopedia-of-Medicine.pdf'}, page_content='Acne— A chronic inflammation of the sebaceous\nglands that manifests as blackheads, whiteheads,and/or pustules on the face or trunk.\nPsoriasis— A skin disorder of chronic, itchy scaling\nmost commonly at sites of repeated minor trauma(e.g. elbows, knees, and skin folds). It affects up to2% of the population in Western countries—malesand females equally.\nRosacea— A chronic inflammation of the face, with'),
 Document(id='1b97b4af-5c6a-4c88-ad65-a6bc8a7211e4', metadata={'page': 54.0, 'source': 'Data\\Encyclopedia-of-Medicine.pdf'}, page_content='(male hormones) cause the glands to produce toomuch sebum. When excess sebum combines withdead, sticky skin cells, a hard plug, or comedo, formsthat blocks the pore. Mild noninflammatory acne con-sists of the two types of comedones, whiteheads andblackheads.\nModerate and severe inflammatory types of acne\

In [36]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500,api_key=OPEN_AI_API_KEY)


In [39]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are assisting for question-answering task. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),

     ]
)

In [40]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input":"What is Acne?"})
print(response["answer"])


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}